In [1]:
import gurobipy as gp

In [2]:
# Sets & Parameters
years = 5
MINE = ['M1', 'M2', 'M3', 'M4']
royalty  = {'M1': 5, 'M2': 4, 'M3': 4, 'M4': 5}
limit    = {'M1': 2.0, 'M2': 2.5, 'M3': 1.3, 'M4': 3.0}
mquality = {'M1': 1.0, 'M2': 0.7, 'M3': 1.5, 'M4': 0.5}
yquality = {1: 0.9, 2: 0.8, 3: 1.2, 4: 0.6, 5: 1.0}

In [3]:
mining = gp.Model("mining")
# Vars
output = mining.addVars(MINE, range(1, years+1), vtype='C', name='output')
total = mining.addVars(range(1, years+1), vtype='C', name='total')
isOperate = mining.addVars(MINE, range(1, years+1), vtype='B', name="isOperate")
isOpen = mining.addVars(MINE, range(1, years+1), vtype='B', name="isOpen")
mining.update()
# Obj.
mining.setObjective(
    gp.quicksum((total[y] * 10) / (1.1 ** (y-1)) for y in range(1, years+1)) -
    gp.quicksum((isOpen[m, y] * royalty[m]) / (1.1 ** (y-1)) for m in MINE for y in range(1, years+1)),
    gp.GRB.MAXIMIZE
)
# s.t.
for m in MINE:
    for y in range(1, years+1):
        mining.addConstr(output[m, y] <= isOperate[m, y] * limit[m], f"upperLimit_{m}_{y}")
        mining.addConstr((years+1-y)*isOpen[m, y] >= gp.quicksum(isOperate[m, i] for i in range(y, years+1)), f"needOpen_{m}_{y}")
for y in range(1, years+1):
    mining.addConstr(gp.quicksum(isOperate[m, y] for m in MINE) <= 3, f"lessThanThree_{y}")
    mining.addConstr(gp.quicksum(output[m, y] for m in MINE) == total[y], f"sumOutput_{y}")
    mining.addConstr(gp.quicksum(output[m,y] * mquality[m] for m in MINE) == total[y] * yquality[y], f"mixQuality_{y}")

# Solve
mining.setParam('OutputFlag', 0)
mining.optimize()
# Result
if mining.status == gp.GRB.OPTIMAL:
    # for v in mining.getVars():
    #     print(f"{v.varName}: {round(v.x, 3)}")
    print(f"Net profit: {round(mining.objVal, 2)} million")
else:
    print("No solution found.")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-05
Net profit: 146.86 million
